# comercio_ext_indices.tb_nomenclatura_brasileira
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NBM_delta`
## 📌 Descrição do arquivo

Tabela de referência com o **código NBM** e sua **descrição textual** do produto.

**NBM (Nomenclatura Brasileira de Mercadorias)** é uma classificação histórica de produtos usada no Brasil antes (ou em paralelo, dependendo do recorte temporal) da consolidação do uso da **NCM**.  
Neste projeto, ela serve como **tabela dimensão/legado** para compatibilização e histórico.

Tipo: Dimensão de produto (legado)  
Uso: Referência / bridge com NCM (via `NBM_NCM.csv`)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NBM`|Código NBM do produto|Identificador do produto na nomenclatura NBM|
|`NO_NBM`|Descrição do produto|Nome/descrição do item conforme NBM|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/nbm/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_nomenclatura_brasileira/"   
val silverTable =    "tb_nomenclatura_brasileira"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType (
  Seq (
    StructField ( "CO_NBM" , StringType ,  nullable = false ) ,
    StructField ( "NO_NBM" , StringType ,  nullable = false ) ,
    StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
    StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
  )
)



## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_NBM", trim(upper(col("CO_NBM").cast(StringType))))
  .withColumn("NO_NBM", col("NO_NBM").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/nbm"))
  .select("CO_NBM","NO_NBM", "TS_REF", "NM_ORIGEM")

###Validações

In [0]:

val dfValid = dfWithDefaults . filter ( 
  col("CO_NBM").isNotNull
)



In [0]:
val dfDedup = dfValid . dropDuplicates (
  Seq (
    "CO_NBM"
  )
)


In [0]:
val dfSilver = spark.createDataFrame(dfDedup.rdd, silverSchema.fieldNames.foldLeft(dfDedup.schema)((schema, name) => schema))

##Carga
> #### **merge**

In [0]:
val deltaTarget = DeltaTable . forName ( "silver_comercio_ext_indices.tb_nomenclatura_brasileira" )

deltaTarget.as("t")
  .merge(
    dfSilver.as("s"),
    """
      t.CO_NBM = s.CO_NBM
    """
  )
  .whenMatched()
  .updateExpr(Map(
    "NO_NBM" -> "s.NO_NBM",
    "TS_REF" -> "s.TS_REF",
    "NM_ORIGEM" -> "s.NM_ORIGEM"
  ))
  .whenNotMatched()
  .insertAll()
  .execute()